# Header for Software License

                                                         
Copyright (C) 2014, Institute for Defense Analyses             
4850 Mark Center Drive, Alexandria, VA; 703-845-2500           
This material may be reproduced by or for the US Government    
pursuant to the copyright license under the clauses at DFARS   
252.227-7013 and 252.227-7014.                                 
                                                               
LARC : Linear Algebra via Recursive Compression                
Authors:                                                       
   * Steve Cuccaro (IDA-CCS)                                    
   * John Daly (LPS)                                            
   * John Gilbert (UCSB, IDA adjunct)                           
   * Jenny Zito (IDA-CCS)                                       
                                                               
Additional contributors are listed in "LARCcontributors".      
                                                               
Questions: larc@super.org                                      
                                                                
All rights reserved.                                       
                                                           
Redistribution and use in source and binary forms, with or  
without modification, are permitted provided that the
following conditions are met:                          
  * Redistribution of source code must retain the above copyright notice, this list of conditions and the following disclaimer. 
  *  Redistribution in binary form must reproduce the above  copyright notice, this list of conditions and the  following disclaimer in the documentation and/or other  materials provided with the distribution.  
  * Neither the name of the copyright holder nor the names of its contributors may be used to endorse or promote products derived from this software without specific prior written permission. 

THIS SOFTWARE IS PROVIDED BY THE COPYRIGHT HOLDERS AND CONTRIBUTORS "AS IS" AND ANY EXPRESS OR IMPLIED WARRANTIES, 
INCLUDING, BUT NOT LIMITED TO, THE IMPLIED WARRANTIES OF MERCHANTABILITY AND FITNESS FOR A PARTICULAR PURPOSE ARE DISCLAIMED.  IN NO EVENT SHALL THE COPYRIGHT HOLDER NOR 
CONTRIBUTORS BE LIABLE FOR ANY DIRECT, INDIRECT, INCIDENTAL, SPECIAL, EXEMPLARY, OR CONSEQUENTIAL DAMAGES (INCLUDING, BUT NOT LIMITED TO, PROCUREMENT OF SUBSTITUTE GOODS OR SERVICES; 
LOSS OF USE, DATA, OR PROFITS; OR BUSINESS INTERRUPTION) HOWEVER CAUSED AND ON ANY THEORY OF LIABILITY, WHETHER IN CONTRACT, 
STRICT LIABILITY, OR TORT (INCLUDING NEGLIGENCE OR  OTHERWISE) ARISING IN ANY WAY OUT OF THE USE OF THIS SOFTWARE, EVEN IF ADVISED OF THE POSSIBILITY OF SUCH DAMAGE.

## Warning!!!

**Due to how Jupyter I/O is implemented, any printout originating from the LARC C code will be directed to the Jupyter console window (i.e., the terminal window that you typed "jupyter notebook" in), instead of in the notebook cell itself.**

In [1]:
import os
import sys 
sys.path.append("../src")
import MyPyLARC as mypy
from ctypes import *
import math
import glob
import json    ## for loading parameter files
import re
import numpy as np
import scipy.stats
import matplotlib.pyplot as plt

In [2]:
level = 10

verbose = 2
LARC_verbose = 0

dim = 2**level
if (verbose > 1):
    print("\nlevel=%d: Program will use up to %d by %d matrices." %(level,dim,dim))
    if ((verbose == 2) or (verbose == 4)):
        print("Verbosity:")
        print("\tThe local verbosity level is verbose=%d and the" %verbose)
        print("\tverbosity level for the LARC package is %d," %LARC_verbose)
        print("\twhere 0=SILENT, 1=BASIC, 2=CHATTY, 3=DEBUG, 4=ALL.")

######################################################
##  General Description of What this Program Does   ## 
######################################################
if (verbose > 1):
    print("\nThis code calculates the 40-th Fibonacci number")
    print("via matrix multiplication.")

######################################################################
# Figure out the scalarType                                          #
# In the Makefile you can compile with different scalarType values   #
# Define string for using in formating filenames                     #
######################################################################
scalarTypeStr = mypy.cvar.scalarTypeStr

######################################################
##   Find out if machine has a large amount of      ##
##   memory available so we can make bigger tables  ##
######################################################
memory_available = mypy.memory_available_GiB()
if (verbose > 0):
    print("\nThe memory available is %ld GiB" %memory_available)
    print("We will use this to select which computing_env to read from parameter file.")
    print("You could write code to select computing_env automatically.")

if (memory_available > 200):
    if (verbose > 0):
        print("\nThis memory is more than 200 GiB\n")
    computing_env = 'large'
else:    
    if (memory_available > 50):
        if (verbose > 0):
            print("\nThis memory is between 50 and 200 GiB\n")
        computing_env = 'medium'
    else:
        if (verbose>0):
            print("\nThis memory is less than 50 GiB\n")
        computing_env = 'small'
        if (memory_available < 10):
            print("\nYou have less than 10 GiB of memory available: defaulting to")
            print("small toy case to avoid overloading your computer\n")
            level = 3
            
if (verbose > 0):
    print("This program believes the computing_environment is %s" %computing_env)

#######################################
##    Print baseline usage report    ##
#######################################
if (verbose > 0):
    print("")
    print("In the following baseline usage report")
    print("RSS, resident set size, refers to size of the process held in RAM.")
    print("HASHSTATS: hash occupancy means, variances and crash resolution chain lengths")
    mypy.memory_and_time_report(0, "stdout")
    print("   SEE JUPYTER CONSOLE FOR USAGE REPORT!")

## read the parameter file into a python dictionary
with open('../InitParams/tutorial.init_params','r') as init_file:
    init_param = json.load(init_file)
    for p in init_param[computing_env]:
        if (verbose > 1):
            print('MatrixExponent: %d' %(p['matrix_exponent']))
            print('OpExponent: %d' %(p['op_exponent']))
            print('MaxLevel: %d' %(p['max_level']))
            print('RegionBitParam: %d' %(p['regionbitparam']))
            print('ZeroRegionBitParam: %d' %(p['zeroregionbitparam']))
            print('ReportIntervalSecs: %d' %(p['report_interval_seconds']))
            print('MinMemRequiredGiB: %d' %(p['min_memGiB_required']))
            print('Verbose: %d' %(p['verbose']))
            print('')
        matrix_exponent = p['matrix_exponent']
        op_exponent = p['op_exponent']
        max_level= p['max_level']
        regionbitparam = p['regionbitparam']
        zeroregionbitparam = p['zeroregionbitparam']
        report_interval_seconds = p['report_interval_seconds']
        min_memGiB_required = p['min_memGiB_required']
        p_verbose = p['verbose']

## warn if the commandline value for LARC_verbose differs from the parameter file value for p_verbose        
if (LARC_verbose > 0):
    if (LARC_verbose != p_verbose):
        print("NOTE: This program uses commandline (LARC_verbose = %d) " %LARC_verbose)
        print("      rather than the parameter file (verbose = %d)." %p_verbose)
        print("      The verbose key is:  0=SILENT, 1=BASIC, 2=CHATTY, 3=DEBUG.")

## initialize LARC
mypy.initialize_larc(matrix_exponent,op_exponent,max_level,regionbitparam,zeroregionbitparam,LARC_verbose)


level=10: Program will use up to 1024 by 1024 matrices.
Verbosity:
	The local verbosity level is verbose=2 and the
	verbosity level for the LARC package is 0,
	where 0=SILENT, 1=BASIC, 2=CHATTY, 3=DEBUG, 4=ALL.

This code calculates the 40-th Fibonacci number
via matrix multiplication.

The memory available is 2 GiB
We will use this to select which computing_env to read from parameter file.
You could write code to select computing_env automatically.

This memory is less than 50 GiB


You have less than 10 GiB of memory available: defaulting to
small toy case to avoid overloading your computer

This program believes the computing_environment is small

In the following baseline usage report
RSS, resident set size, refers to size of the process held in RAM.
HASHSTATS: hash occupancy means, variances and crash resolution chain lengths
   SEE JUPYTER CONSOLE FOR USAGE REPORT!
MatrixExponent: 22
OpExponent: 19
MaxLevel: 8
RegionBitParam: -1
ZeroRegionBitParam: -1
ReportIntervalSecs: 180
Min

In [3]:
######################################################################
# Step 1. Construct the following Fibonacci matrix:
#   [ 0 1 ]
#   [ 1 1 ]
######################################################################

# Define the matrix elements and sizes.
fibMatrixElts = [0, 1, 1, 1]
row_level = 1
num_rows = 2**row_level
col_level = 1
num_cols = 2**col_level

# Convert the elements to strings for passing to MyPyLARC.
fibMatrixStr = mypy.map_to_str(fibMatrixElts, scalarTypeStr)

# Call the MyPyLARC matrix construction from elements routine.
fibMatID = mypy.row_major_list_to_store(fibMatrixStr, row_level, col_level, num_cols)

In [4]:
# Print out the matrix from the previous cell.
# Unfortunately, due to how Jupyter is implemented,
# the printout will be directed to the Jupyter console
# window instead of below.
mypy.print_naive(fibMatID)
print("   SEE JUPYTER CONSOLE FOR MATRIX OUTPUT!")

   SEE JUPYTER CONSOLE FOR MATRIX OUTPUT!


In [5]:
# To calculate the n-th Fibonacci number,
# raise the Fibonacci matrix to the (n-1)
# power, and then look at the lower right
# corner element.
n = 40

fibPowerMatID = mypy.get_identity_pID(row_level)
for i in range(n-1):
    fibPowerMatID = mypy.matrix_mult(fibPowerMatID, fibMatID)

# Print out the power matrix to the Jupyter console
mypy.print_naive(fibPowerMatID)
print("   SEE JUPYTER CONSOLE FOR MATRIX OUTPUT!")

# Obtain the [1,1] element of the power matrix
answer = mypy.get_readableString_scalar_from_pID_and_coords(fibPowerMatID, 1, 1)
print("The answer is: ", answer)

   SEE JUPYTER CONSOLE FOR MATRIX OUTPUT!
The answer is:  102334155+I*0
